### Span detection using BERT
It is kind of extractive summerization in Q/A systems

In [ ]:
pip install transformers torch


In [ ]:
# Using Bert for span detection task. Note that, we use special model of large BERT for QA
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load pre-trained BERT model and tokenizer for question answering
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
model = BertForQuestionAnswering.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [4]:
# Define the question and the context
question = "Who wrote Romeo and Juliet?"
context = "Romeo and Juliet is a tragedy written by William Shakespeare early in his career about two young star-crossed lovers."

# Tokenize the input, add special tokens ([CLS] and [SEP]), and convert to tensor
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"]
inputs

{'input_ids': tensor([[  101,  2040,  2626, 12390,  1998, 13707,  1029,   102, 12390,  1998,
         13707,  2003,  1037, 10576,  2517,  2011,  2520,  8101,  2220,  1999,
          2010,  2476,  2055,  2048,  2402,  2732,  1011,  4625, 10205,  1012,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [5]:
# Get the predicted start and end positions
with torch.no_grad():
    res = model(input_ids)

start_pos = torch.argmax(res.start_logits)
end_pos = torch.argmax(res.end_logits)

print("The probabilities for start are:", res.start_logits)
print("The strat index is:", start_pos)


The probabilities for start are: tensor([[-6.3058, -5.7577, -6.7796, -3.3953, -8.1165, -6.3404, -8.4627, -6.3057,
          0.1943, -5.3829, -3.9351, -5.0365, -3.2273, -2.6362, -1.4974, -2.2578,
          7.5604,  3.0763, -4.2326, -6.6545, -3.2653, -4.8068, -7.0875, -6.4545,
         -6.8650, -6.7581, -7.6815, -7.6125, -5.6213, -6.3056, -6.3057]])
The strat index is: tensor(16)


In [6]:
# Inference
# Convert the token IDs back to tokens and join to get the answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_pos:end_pos+1]))
print(answer)

william shakespeare


In [14]:
# Another Example



question = 'what services does Coles provied?'

context = "Coles Supermarkets Australia Pty Ltd, trading as Coles, is an Australian supermarket, retail and consumer services chain,\
             headquartered in Melbourne as part of the Coles Group. Founded in 1914 in Collingwood by George Coles, Coles operates 846[3] \
             supermarkets throughout Australia, including several now re-branded Bi-Lo Supermarkets. Coles has over 120,000 employees[3][4]\
              and accounts for around 27 per cent of the Australian market.[5] Coles' large head office site in Melbourne's inner south-east \
              has 4,000 employees of the workforce located inside.\
                Coles Online is the company's online shopping ('click & collect' and home delivery) service.\
              Between 1986 and 2006, Coles Supermarkets was a brand of Coles Myer, later Coles Group, prior to Wesfarmers purchasing Coles \
              Group in 2007. It became a subsidiary of Coles Group again after Wesfarmers spun-off the business in November 2018.[6]\
              In 2020, Coles changed its slogan to 'Value the Australian way'"

# Tokenize the input, add special tokens ([CLS] and [SEP]), and convert to tensor
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
input_ids = inputs["input_ids"]


# Get the predicted start and end positions
with torch.no_grad():
    res = model(input_ids)

start_pos = torch.argmax(res.start_logits)
end_pos = torch.argmax(res.end_logits)



# Inference
# Convert the token IDs back to tokens and join to get the answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_pos:end_pos+1]))
print(answer)


consumer services


### Abstractive Summerization for QA

In [16]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [4]:
# Define the context and the question
questions = ['what is Coles?',
            'What does it do?',
            'When it is established?',
            'Is Mayer mentioned in the cotext (Yes/NO)? ',
            'Summerize the context'
]

context = "Coles Supermarkets Australia Pty Ltd, trading as Coles, is an Australian supermarket, retail and consumer services chain,\
             headquartered in Melbourne as part of the Coles Group. Founded in 1914 in Collingwood by George Coles, Coles operates 846[3] \
             supermarkets throughout Australia, including several now re-branded Bi-Lo Supermarkets. Coles has over 120,000 employees[3][4]\
              and accounts for around 27 per cent of the Australian market.[5] Coles' large head office site in Melbourne's inner south-east \
              has 4,000 employees of the workforce located inside.\
                Coles Online is the company's online shopping ('click & collect' and home delivery) service.\
              Between 1986 and 2006, Coles Supermarkets was a brand of Coles Myer, later Coles Group, prior to Wesfarmers purchasing Coles \
              Group in 2007. It became a subsidiary of Coles Group again after Wesfarmers spun-off the business in November 2018.[6]\
              In 2020, Coles changed its slogan to 'Value the Australian way'"


# Format the input string: The convention for T5 is to prepend "question: [your question] context: [your context]" It is like prompting
for question in questions:
  input_str = f"question: {question} context: {context}"
  inputs = tokenizer.encode(input_str, return_tensors="pt", max_length=512, truncation=True)

  answer_ids = model.generate(inputs, max_length=500, num_beams=4, early_stopping=True)
  # print(answer_ids)

  # Decode and print the answer
  answer = tokenizer.decode(answer_ids[0], skip_special_tokens=True)
  print(f"Question: {question}\nAnswer: {answer}\n")

Question: what is Coles?
Answer: an Australian supermarket, retail and consumer services chain

Question: What does it do?
Answer: operates 846[3] supermarkets throughout Australia

Question: When it is established?
Answer: 1914

Question: Is Mayer mentioned in the cotext (Yes/NO)? 
Answer: yes

Question: Summerize the context
Answer: Coles Supermarkets Australia Pty Ltd, trading as Coles, is an Australian supermarket, retail and consumer services chain, headquartered in Melbourne

